In [1]:
import sys
del sys.path[9]
del sys.path[8]
del sys.path[7]
del sys.path[5]
del sys.path[6]
import numpy as np

sys.path.insert(0,'/home/pgaemers/software/epix') # 'fast_sim_test' branch

import epix
print(epix.__version__)

import straxen
from epix.simulator.fast_simulator import StraxSimulator
import numba

/dali/lgrandi/strax/straxen/straxen/rucio.py:29: UserWarning: No installation of rucio-clients found. Can't use rucio remote backend.
  warnings.warn("No installation of rucio-clients found. Can't use rucio remote backend.")


Using nestpy version 1.3.0
0.1.2


In [2]:
raw_data_dir='/dali/lgrandi/pkavrigin/'
raw_data_filename='ambe_tut_cw5900mm.root'

fast_sim_output_path= '/dali/lgrandi/pgaemers/strax_data_epix'

In [3]:
st = straxen.contexts.xenonnt_simulation(output_folder=fast_sim_output_path)
run_id = '1'

fax_config='fax_config_nt_design.json'

# epix config
epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':1000,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.00001,
           'tag_cluster_by':'time',
           'max_delay':1e7,
           'source_rate':-1}

# configuration files; to be moved into the plugin
configuration_files={'nv_pmt_qe':'nveto_pmt_qe.json',
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_relative_lce_map':straxen.pax_file('XENON1T_s1_xyz_lce_true_kr83m_SR1_pax-680_fdc-3d_v0.json'),
                     's2_xy_correction_map':straxen.pax_file('XENON1T_s2_xy_ly_SR1_v2.2.json'),}

st.config.update(dict(fax_config=fax_config,
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      configuration_files=configuration_files,
                      fax_config_overrides= {'detector':'XENONnT',
                                             "enable_field_dependencies": {
                                             "survival_probability_map": False,
                                             "drift_speed_map": False,
                                             "diffusion_longitudinal_map": False,
                                             "diffusion_radial_map": False,
                                             "diffusion_azimuthal_map": False}}))

In [4]:
st.register(StraxSimulator)

epix.simulator.fast_simulator.StraxSimulator

In [5]:
st.make(run_id,'events_tpc')

Removing data in /dali/lgrandi/pgaemers/strax_data_epix/1-events_tpc-b3gc5337ra to overwrite
Removing data in /dali/lgrandi/pgaemers/strax_data_epix/1-events_nveto-b3gc5337ra to overwrite


epix configuration:  {'path': '/dali/lgrandi/pkavrigin/', 'file_name': 'ambe_tut_cw5900mm.root', 'debug': True, 'entry_start': 0, 'entry_stop': 1000, 'cut_by_eventid': False, 'micro_separation_time': 10.0, 'micro_separation': 1e-05, 'tag_cluster_by': 'time', 'max_delay': 10000000.0, 'source_rate': -1, 'detector_config': [<epix.detector_volumes.SensitiveVolume object at 0x7fc964a645b0>, <epix.detector_volumes.SensitiveVolume object at 0x7fc964a644c0>], 'outer_cylinder': {'max_z': 7.3936, 'min_z': -154.6555, 'max_r': 66.4}}
Total entries in input file = 25937
Starting to read from output file entry 0
Ending read in at output file entry 1000
It took 9.1800 sec to load data.
Finding clusters of interactions with a dr = 1e-05 mm and dt = 10.0 ns
It took 5.4908 sec to find clusters.
It took 6.5248 sec to merge clusters.
Removing clusters not in volumes: TPC BelowCathode
Number of clusters before: 60207
Number of clusters after: 58444
Assigning electric field to clusters
Generating photons an

/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: divide by zero encountered in func (vectorized)
  outputs = ufunc(*inputs)
/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: overflow encountered in func (vectorized)
  outputs = ufunc(*inputs)
/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


It took 4.0664 sec to get quanta.
Clean event separation
Min. S2 amp BEFORE MACRO-CLUSTERING: 1


Exception in thread CRITICAL:ThreadedMailboxProcessor:Target Mailbox (events_tpc) killed, exception <class 'strax.mailbox.MailboxKilled'>, message (<class 'IndexError'>, IndexError('index 106872 is out of bounds for axis 0 with size 106872'), <traceback object at 0x7fc9642219c0>)
divide_outputs:events_tpc:
Traceback (most recent call last):
  File "/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/dali/lgrandi/strax/strax/strax/mailbox.py", line 267, in _send_from
    self.kill_from_exception(e)
  File "/dali/lgrandi/strax/strax/strax/mailbox.py", line 186, in kill_from_exception
    raise e
  File "/dali/lgrandi/strax/strax/strax/mailbox.py", line 254, in _send_from
    x = next(iterable)
  File "/dali/lgrandi/strax/strax/strax/plugin.py", line 394, in iter
    yield self

IndexError: index 106872 is out of bounds for axis 0 with size 106872

In [69]:
@numba.njit
def _merge_these_clusters(s2_area1, z1, s2_area2, z2):
    sensitive_volume_ztop=0 #it's the ground mesh, the top liquid level is at 2.7; // mm
    max_s2_area = max(s2_area1,s2_area2)
    if max_s2_area>5000:
        SeparationDistanceIntercept = 0.00024787 * 5000. + 3.4056346550312973
        SeparationDistanceSlope = 5.5869678412887262e-07 * 5000. + 0.0044792968
    else:
        SeparationDistanceIntercept = \
          0.00024787 * max_s2_area + 3.4056346550312973
        SeparationDistanceSlope = \
          5.5869678412887262e-07 * max_s2_area + 0.0044792968
    SeparationDistance = \
        SeparationDistanceIntercept - \
        SeparationDistanceSlope * (-sensitive_volume_ztop +\
            (z1 + z2) * 0.5)
    return z1-z2<SeparationDistance

@numba.njit
def macro_cluster_events(instructions):
    """Loops over all instructions, checks if it's an s2 and if there is another s2 within the same event
        within the macro cluster distance, if it is they are merged."""
    for ix1 in range(len(instructions)):
        if instructions[ix1]['type']!=2:
            continue
        for ix2 in range(1,len(instructions[ix1:])):
            if instructions[ix1]['event_number']!=instructions[ix1+ix2]['event_number']:
                break
            if instructions[ix1+ix2]['type']!=2:
                continue
            if _merge_these_clusters(instructions[ix1]['amp'],instructions[ix1]['z'],
                                     instructions[ix1+ix2]['amp'],instructions[ix1+ix2]['z']):
                instructions[ix1+ix2]['x'] = (instructions[ix1]['x']+instructions[ix1+ix2]['x'])*0.5
                instructions[ix1+ix2]['y'] = (instructions[ix1]['y']+instructions[ix1+ix2]['y'])*0.5
                instructions[ix1+ix2]['z'] = (instructions[ix1]['z']+instructions[ix1+ix2]['z'])*0.5
                instructions[ix1+ix2]['amp'] = int((instructions[ix1]['amp']+instructions[ix1+ix2]['amp']))
                instructions[ix1]['amp']=-1 #flag to throw this instruction away later
                break

In [70]:
instructions=np.load('instructions.npy')
instructions=instructions[instructions['type']==2]

In [71]:
instructions[instructions['event_number']==165]

array([(165, 2, 1826000000,  20.039852 , -31.576733, -49.366394, 5, 0, 5.582118 , 1153, 1),
       (165, 2, 1826000023,   5.42555  , -34.919178, -47.077187, 4, 0, 5.6402416, 1153, 1),
       (165, 2, 1826000038,  14.033266 , -38.671368, -46.949444, 1, 0, 2.728987 , 1153, 1),
       (165, 2, 1826000046,  13.655611 , -40.928158, -42.645508, 2, 0, 5.1514997, 1153, 1),
       (165, 2, 1826000063,   6.921006 , -40.997864, -50.967632, 2, 0, 3.1622007, 1153, 1),
       (165, 2, 1826000112, -14.90233  , -33.180733, -32.15068 , 1, 0, 5.9501677, 1153, 1),
       (165, 2, 1826000162,   5.78072  , -44.21184 , -17.31942 , 1, 0, 1.6405127, 1153, 1),
       (165, 2, 1826000186,   5.8595104, -39.100216,  -3.758201, 2, 0, 2.4491782, 1153, 1)],
      dtype=[(('Waveform simulator event number.', 'event_number'), '<i4'), (('Quanta type (S1 photons or S2 electrons)', 'type'), 'i1'), (('Time of the interaction [ns]', 'time'), '<i8'), (('X position of the cluster[cm]', 'x'), '<f4'), (('Y position of the clus

In [72]:
macro_cluster_events(instructions)

In [73]:
instructions[instructions['event_number']==165]

array([(165, 2, 1826000000, 20.039852 , -31.576733, -49.366394, -1, 0, 5.582118 , 1153, 1),
       (165, 2, 1826000023, 12.732701 , -33.247955, -48.22179 , -1, 0, 5.6402416, 1153, 1),
       (165, 2, 1826000038, 13.382984 , -35.959663, -47.585617, -1, 0, 2.728987 , 1153, 1),
       (165, 2, 1826000046, 13.519298 , -38.44391 , -45.115562, -1, 0, 5.1514997, 1153, 1),
       (165, 2, 1826000063,  6.921006 , -40.997864, -50.967632, -1, 0, 3.1622007, 1153, 1),
       (165, 2, 1826000112,  3.114745 , -38.40509 , -44.800377, -1, 0, 5.9501677, 1153, 1),
       (165, 2, 1826000162,  4.4477324, -41.308464, -31.059898, -1, 0, 1.6405127, 1153, 1),
       (165, 2, 1826000186,  5.1536217, -40.204338, -17.40905 , 18, 0, 2.4491782, 1153, 1)],
      dtype=[(('Waveform simulator event number.', 'event_number'), '<i4'), (('Quanta type (S1 photons or S2 electrons)', 'type'), 'i1'), (('Time of the interaction [ns]', 'time'), '<i8'), (('X position of the cluster[cm]', 'x'), '<f4'), (('Y position of the clus

In [45]:
bla

-1